# BS Optimization

- This script will take a fake BS and perform an optimization to determine highest profit generating balances
- After the first successful run, I will layer on additional complexity in terms of contraints

In [25]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [2]:
# Define objective function
# Only used in the version of that uses the 'minimize' function

# def fun(x):
    
#     sva = x * nsi
    
#     return -sva

In [37]:
# Define starting balances
# Not need for linprog implementation

# x0 = np.array(df['start'])

# x0.reshape(-1, 1)

In [32]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=26)

In [33]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,srwa_min,gsib
Index,,,,,,,,,,
1,prime_auto,10000,5000,-3000,80,1,0.2,1.0,0.115,NaN
2,subprime_auto,2000,2000,-500,150,1,0.6,1.0,0.115,NaN
3,mtg_30_fixed,40000,20000,-10000,70,1,0.3,0.5,0.115,NaN
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0.115,NaN
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0.115,NaN


In [30]:
# Define objective function coefficients
# We are 'minimizing' the spread so will make this negative

c = -np.array(df['spread'])

## Inequality Constraint

- The inequality constraint for our optimization is the the SRWA % * 11% * the balance - equity needs to be greater than or equal to 0.
- Since it is greater than, for purposes of Scipy notation, we need to flip the sign

In [31]:
# Define the CET1 minimum

CET1_min = 0.11 

In [44]:
# Multiply the RWA % by the RWA minimum and flip the sign

A_ineq = [list(-(df['s_rwa'] * df['srwa_min']))]

# Define the other side of the inequality equation (we want to be at least at the minimum, so this is 0)

b_ineq = [0]

## Equality Constraint

- This constraint says that Assets - Liabilities must equal to 0.
- I've pre-programmed this to an extent by including a assets (+100%) and liability (-100%) weight in the inputs.

In [46]:
# Create Asset and Liability equality constraint.

A_eq = [list(df['A_L'])]

In [23]:
# Equality constraint vector defined below. This is saying that assets must equal liabilities on the B/S

a_l_constraint = 0

b_eq = [a_l_constraint]

In [6]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [16]:
# Define bounds in format needed for linprog

bounds = list(df['bounds'])

## Optimize B/S

In [52]:
res = spo.linprog(c, A_ub = A_ineq, b_ub = b_ineq, A_eq = A_eq, b_eq = b_eq, bounds = bounds, method='revised simplex')

## Results

In [56]:
# This shows that a solution was found

res.success

True

In [57]:
# Show the output of the objective function

res.fun

-215653500.0

In [60]:
# Compare to starting profitability

start_profitability = -sum(df['start'] * df['spread'])

(res.fun - start_profitability) / start_profitability

# Profitability doubled!

1.0300621293419938

In [61]:
# Show ending balances

res.x

array([ 15000.,   4000.,  60000.,  23000.,   7500.,  30000.,  13500.,
        70000.,  17500.,  35000.,  50000.,  60000., 110000., 105000.,
        75000.,  95000.,  34000.,   1500., 235000.,  85000., 130000.,
        35000., 125500.,  62000., 136000.,  -2500.])

In [63]:
# Append ending balance to our data and compare growth vs. shrink

df['optimal_balance'] = res.x

df['balance_change'] = df['optimal_balance'] - df['start']

In [64]:
balance_results = df[['Product', 'balance_change']]

In [65]:
# Aha, this shows that something is wrong in the formula of our optimization.

balance_results

,Product,balance_change
Index,,
1,prime_auto,5000.0
2,subprime_auto,2000.0
3,mtg_30_fixed,20000.0
4,mtg_15_fixed,8000.0
5,mtg_7_fixed,2500.0
6,mtg_15_arm,10000.0
7,mtg_7_arm,3500.0
8,consumer_card,20000.0
9,business_card,2500.0
